In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.cluster import DBSCAN
from torch import nn

In [8]:
%cd ../

/home/tugash/Documents/gt-explorer


In [9]:
device = torch.device('cpu')

In [10]:
data = torch.load("data/processed/mini_dataset.pt", device)

In [11]:
data.shape

torch.Size([5000, 60483])

In [12]:
data_y = torch.load("data/processed/mini_data_y.pt", device)

In [14]:
data_set = TensorDataset(data.float(), data_y.long())

In [15]:
train_batches = DataLoader(data_set, batch_size=500, shuffle=True)

In [16]:
sum(data.var(dim=0) < 1e-2)

tensor(17300)

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(60483, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
#         x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [18]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=60483, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [19]:
logits = model(data.float())
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8, 2, 4,  ..., 4, 4, 4])


In [20]:

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 60483]) | Values : tensor([[ 0.0003, -0.0019,  0.0023,  ..., -0.0005, -0.0008,  0.0001],
        [-0.0011, -0.0012, -0.0028,  ..., -0.0029, -0.0039,  0.0039]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0015, -0.0012], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0065, -0.0268,  0.0223,  ...,  0.0310,  0.0057,  0.0159],
        [ 0.0158,  0.0378,  0.0025,  ..., -0.0288, -0.0077,  0.0025]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([ 0.0359, -0.0264], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0274, -0.0094,  0.0440,  ...,  0.0104,  0.0414, -0.0063],
        [ 0.0030, -0.0365, -0.0108,  ...,  0.0364, -0.0175,  0.0020]],
       grad_fn=<SliceBackward>) 



In [21]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_batches, model, loss_fn, optimizer)
#     test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


IndexError: Target 15 is out of bounds.